In [1]:
from syft import Worker, MessageStatus
from syft.core.node.new.project import Project
import syft as sy

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


In [2]:
worker = Worker.named("test-domain-1", processes=1, reset=False)

SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.Data

In [3]:
domain_client = worker.guest_client.login(email="info@openmined.org", password="changethis")

# Get messages

In [4]:
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)

In [5]:
messages

,type,id,subject,status,created_at,linked_obj
0,syft.core.node.new.messages.Message,6a722d6815f04d808463b509181c8342,Project Approval,MessageStatus.UNDELIVERED,2023-04-17 14:02:18,<<class 'syft.core.node.new.project.Project'>:...
1,syft.core.node.new.messages.Message,c3d5d55d387b4225b36d9e4daf9c97c5,Approval Request,MessageStatus.UNDELIVERED,2023-04-17 14:02:18,<<class 'syft.core.node.new.request.Request'>:...


In [6]:
project_message = [x for x in messages if issubclass(x.linked_obj.object_type, Project)][0]

In [7]:
request = project_message.link.requests[0]
func = request.changes[0].link
op = func.output_policy_type

In [8]:
func

```python
class UserCode:
  id: str = c1e78985a1624ad1946935c425576786
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  user_verify_key: str = fff9d7a435413560f741bb638418a463971f4c218b048207b0c57164e2284a64
  raw_code: str = "@sy.syft_function(input_policy=sy.ExactMatch(bikes=asset.mock),
                  output_policy=sy.SingleExecutionExactOutput())
def get_bike_hist(bikes):
    berri_bikes = bikes[['Berri 1']].copy()
    berri_bikes.loc[:,'weekday'] = berri_bikes.index.weekday
    weekday_counts = berri_bikes.groupby('weekday').aggregate(sum)
    return weekday_counts
"
  input_policy_type: str = <class 'syft.core.node.new.policy.ExactMatch'>
  input_policy_init_kwargs: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): {'bikes': <UID: cf79a8cf636f4c1a90835d976c28831f>}}
  input_policy_state: str = b''
  output_policy_type: str = <class 'syft.core.node.new.policy.OutputPolicyExecuteOnce'>
  output_policy_init_kwargs: str = {}
  output_policy_state: str = b''
  parsed_code: str = "

def user_func_get_bike_hist_fff9d7a435413560f741bb638418a463971f4c218b048207b0c57164e2284a64_8d8256c2fdc5571401947971c192627c1969ce527c7715d90da86fa9f28c9558(bikes):

    def get_bike_hist(bikes):
        berri_bikes = bikes[['Berri 1']].copy()
        berri_bikes.loc[(:, 'weekday')] = berri_bikes.index.weekday
        weekday_counts = berri_bikes.groupby('weekday').aggregate(sum)
        return weekday_counts
    result = get_bike_hist(bikes=bikes)
    return result
"
  service_func_name: str = "get_bike_hist"
  unique_func_name: str = "user_func_get_bike_hist_fff9d7a435413560f741bb638418a463971f4c218b048207b0c57164e2284a64_8d8256c2fdc5571401947971c192627c1969ce527c7715d90da86fa9f28c9558"
  user_unique_func_name: str = "user_func_get_bike_hist_fff9d7a435413560f741bb638418a463971f4c218b048207b0c57164e2284a64"
  code_hash: str = "8d8256c2fdc5571401947971c192627c1969ce527c7715d90da86fa9f28c9558"
  signature: str = (bikes)
  status: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>}
  input_kwargs: str = ['bikes']
  enclave_metadata: str = None

```

In [9]:
get_col_user_function = func.unsafe_function

In [10]:
real_data = domain_client.datasets[0].assets[0].data

In [11]:
real_result = get_col_user_function(bikes=real_data)

In [12]:
bikes=real_data
berri_bikes = bikes[['Berri 1']].copy()
berri_bikes.loc[:,'weekday'] = berri_bikes.index.weekday
weekday_counts = berri_bikes.groupby('weekday').aggregate(sum)
real_result = weekday_counts

In [13]:
real_result[:3]

,Berri 1
weekday,
0,134298
1,135305
2,152972


In [14]:
result = request.accept_by_depositing_result(real_result)
result
assert isinstance(result, sy.SyftSuccess)